# BERT Test Implementation 

@authors: taiyyoson, ebeckk

In [1]:
!pip install transformers datasets scikit-learn torch

  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 17.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 16.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.9/30.9 MB 16.4 MB/s eta 0:00:0000:0100:01
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 9.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 11.4 MB/s eta 0:00:00a 0:00:01
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)


In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

In [3]:
## sample test data to run BERT on
data = {
    "text": ["I love this movie!", "This is terrible.", "Amazing experience!", "Worst day ever.", "So good!", "Really bad."],
    "label": [1, 0, 1, 0, 1, 0]  # 1 = Positive, 0 = Negative
}

df = pd.DataFrame(data)

## split data
train_X, test_X_val, train_y, test_y = train_test_split(df["text"], df["label"], test_size=0.2)

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

## tokenize the texts w tokenizer
train_encodings = tokenizer(list(train_X), truncation=True, padding=True)
val_encodings = tokenizer(list(test_X_val), truncation=True, padding=True)